# Decision Tree Assignment2.

In [1]:
import pandas as pd 
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
df.drop("RID", axis=1, inplace = True) #RID는 그냥 Index라서 삭제

In [2]:
df

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## 함수 만들기

In [3]:
def getEntropy(df, feature) :
    
    probability = df[feature].value_counts()/df.shape[0] # feature의 비율
    probability = probability.values # np.array로 변환

    entropy = 0 # 누적해서 더해나가기 때문에 0으로 설정
    for p in probability:
        entropy += -(p*np.log2(p)) # entropy 구하기
        
    return(entropy)

In [4]:
getEntropy(df, "class_buys_computer")

0.9402859586706311

---

In [5]:
def getGainA(df, target) :
    info_D = getEntropy(df, target) # 목표변수 Feature에 대한 Info(Entropy)를 구한다.
    columns = list(df.loc[:, df.columns != target]) # 목표변수를 제외한 나머지 설명변수들을 리스트 형태로 저장한다.
    
    gains = [] # feature 별 information gain을 저장할 리스트
    for col in columns: # columns에 대해서 반복문
        entropy_label = [] # feature의 unique한 값에 대해서 각 entropy (= sum(-p*log2(p)))
        feature_label = [] # feature의 unique한 값의 비율
        
        col_unique = df[col].unique() # feature의 unique한 값 추출
        for col_value in col_unique: # feature의 unique한 값에 대해서 반복문
            index = (df[col] == col_value) # feature의 unique에 대한 데이터프레임을 추출하기 위해 index 정보 저장
            feature_label.append(sum(index)/len(index)) # feature의 unique한 값의 비율 구해서 feature_label에 추가
            entropy_label.append( getEntropy(df.loc[df[col] == col_value, [target]], target) ) # feature의 unique한 값에 대해서 entropy 구하고 추가

        info_feature = 0
        for value in zip(feature_label, entropy_label):
            info_feature += value[0]*value[1] # info(feature)를 구하는 과정

        gains.append(info_D - info_feature) # feature의 information gain을 구하고 추가
     
    result = dict(zip(columns, gains)) # 각 변수에 대한 Information Gain 을 Dictionary 형태로 저장한다.
    return(result)

In [6]:
getGainA(df, "class_buys_computer") 

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}

## 결과 확인하기

In [7]:
my_dict = getGainA(df, "class_buys_computer")
def f1(x):
    return my_dict[x]
key_max = max(my_dict.keys(), key=f1)
print('정보 획득이 가장 높은 변수는', key_max, "이며 정보 획득량은", my_dict[key_max], "이다.")

정보 획득이 가장 높은 변수는 age 이며 정보 획득량은 0.24674981977443933 이다.
